In [8]:
import sympy as sp

# Variables
T, P = sp.symbols('T P')
n = sp.IndexedBase('n')
i, k = sp.symbols('i k', integer=True)

# Definimos φ_k como función general
phi = sp.Function('phi')

# Expresión: φ_k = phi(n[k], T, P)
phi_k = phi(n[k], T, P)

# Derivar respecto a n[i]
dphi_dni = sp.diff(phi_k, n[i])


In [9]:
phi_k

phi(n[k], T, P)

In [14]:
dphi_dni.subs(i, k)

Derivative(phi(n[k], T, P), n[k])

## Ejemplo total

In [38]:
# Variable dummy Nc (numero de componentes)
nc = sp.symbols("Nc", integer=True)

# Indices
# Para derivadas de n
i = sp.symbols("i", cls=sp.Idx)
j = sp.symbols("j", cls=sp.Idx)

# Indices para variables
k = sp.symbols("k", cls=sp.Idx)
l = sp.symbols("l", cls=sp.Idx)

# Variables
n = sp.IndexedBase("n", shape=(nc,))
t = sp.symbols("T")
v = sp.symbols("V")


r = sp.IndexedBase('r')
q = sp.IndexedBase('q')

a = sp.IndexedBase('a')
b = sp.IndexedBase('b')

# Definimos φ_k como función general
phi = sp.Function('phi')

# Expresión: φ_k = phi(n[k], T, P)
phi_k = phi(n[k])

In [ ]:
def simple_kronecker(expr):
    solved = sp.diff(expr, n[i])
    
    solved = solved.doit().subs({i >= 1: True, nc >= i: True})

    return solved

In [ ]:
# Subclase de Function
class CFunction(sp.Function):
    @classmethod
    def eval(cls, n_k, T):
        # No simplificamos nada en eval
        pass

    def fdiff(self, argindex=1):
        raise NotImplementedError("Use .diff(var) for partial derivatives.")

    def diff(self, sym, **kwargs):
        # Solo tratamos derivadas respecto a elementos de n[i]
        if isinstance(sym, sp.Indexed) and sym.base == n:
            k = self.args[0].args[0]  # Extrae el índice k de n[k]
            i = sym.args[0]           # Extrae el índice i de n[i]
                       
            return CFunction(*self.args[:], name=r'\frac{\partial \phi}{\partial n}')
        else:
            # Por defecto: usa el método base
            return super().diff(sym, **kwargs)
        
    def __new__(cls, *args, name=None, **kwargs):
        obj = super().__new__(cls, *args, **kwargs)
        obj._custom_name = name or cls.__name__
        return obj

    def __str__(self):
        return f"{self._custom_name}({', '.join(map(str, self.args))})"

    __repr__ = __str__
    
    def _sympystr(self, printer):
        return f"{self._custom_name}({', '.join(printer._print(arg) for arg in self.args)})"

    def _latex(self, printer):
        return f"{self._custom_name}\\left({', '.join(printer._print(arg) for arg in self.args)}\\right)"


k = sp.Symbol('k', integer=True)
phi_k = CFunction(n[k], T, name=r'\phi')

phi_k

\phi(n[k], T)

In [61]:
exp = sp.Sum(n[k] * phi_k, (k, 0, nc))

exp

Sum(\phi(n[k], T)*n[k], (k, 0, Nc))

In [63]:
coso = sp.diff(exp, n[i])

In [64]:
coso

Sum(CFunction(n[k], T)*KroneckerDelta(k, i) + CFunction(n[k], T)*n[k], (k, 0, Nc))